In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import re

# Socrata specific 
socrata_domain = "data.cityofnewyork.us"
socrata_token = "F52GVJzdJf2mpjl7n17oXvzqF" # should be an environment variable
dataset_id = "fhrw-4uyv"

# set up socrate client endpoint
client = Socrata(socrata_domain, socrata_token)

# Data retrieval and cleaning

In [ ]:
# get the total number of complaints made in 2017
num_complaints_2017 = client.get(dataset_id,
                    select="count(*)",
                    where="created_date between '2017-01-01T00:00:00.001' and '2017-12-31T23:59:59'")[0]['count']

print("Number of complaints in 2017: {}".format(num_complaints_2017))

Number of complaints in 2017: 2445387


In [ ]:
# get only the necessary fields and the exact amount of complaints made in 2017
# starting from the 1st second into 2017 because there are about 50 complaints at time 00:00:00 and must be bogus
results = client.get(dataset_id,
                    select="unique_key, created_date, complaint_type, descriptor, incident_zip, borough, city",
                    where="created_date between '2017-01-01T00:00:00.001' and '2017-12-31T23:59:59'",
                    order="created_date ASC",
                    limit=num_complaints_2017)

# Convert to pandas DataFrame
complaints_df_ = pd.DataFrame.from_records(results)

In [ ]:
complaints_df = complaints_df_.copy()

##### Clean up zip codes by removing any null/invalid ones and truncating zip+4 digits

In [ ]:
# returns only first 5 digits of any zipcode greater than 0
def keep_five(zipc):
    m = re.search('\d{5}', zipc)
    return m[0] if m and int(m[0]) > 0 else None

# remove rows with null zip codes
complaints_df = complaints_df[pd.notnull(complaints_df['incident_zip'])]

# keep rows with valid zip codes and truncating 5+4 zips to 5 digits only
complaints_df['incident_zip'] = complaints_df['incident_zip'].apply(keep_five)

# remove rows with 'None' zip codes
complaints_df = complaints_df[~complaints_df['incident_zip'].isin([None])]

In [ ]:
# show number of complaints after cleaning zip codes
complaints_before = complaints_df_.shape[0]
complaints_after = complaints_df.shape[0]

unspecified_boroughs_before = complaints_df_.borough.value_counts()['Unspecified']
unspecified_boroughs_after = complaints_df.borough.value_counts()['Unspecified']

print("Number of rows removed by cleaning zip codes (removing null and invalid): {}".format(complaints_before - complaints_after))
print("Number of unspecified boroughs removed by cleaning zip codes : {}".format(unspecified_boroughs_before - unspecified_boroughs_after))

### Scrape website to get accurate zip code to borough info
#### Use the zip codes found on the website to replace 311 data (5513 rows being updated)

In [ ]:
# Use beautifulsoup to scrape webpage for zip codes and their boroughs
# The website used only has 240 zip codes mapped to boroughs while the 311 data has ~600 unique zip codes
# The website data is incomplete but still helpful in filtering 
import requests
from bs4 import BeautifulSoup

# create a mapping for each zipcode and its corresponding borough
zip_borough_web = {}

# fetch website to scrape
url = "https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php"
page = requests.get(url)

# create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')
ziptable = soup.find('table')

# iterate over all rows in table
for row in ziptable.findAll('tr'):
    data = row.findAll('td')
    
    # The structure of the website zip code table. each tr has 2 zips and boroughs
    zip1 = data[0].text.strip()
    borough1 = data[1].text.upper().strip()
    zip2 = data[3].text.strip()
    borough2 = data[4].text.upper().strip()
    
    if borough1 in 'STATEN':
        borough1 += ' ISLAND'
    
    if borough2 in 'STATEN':
        borough2 += ' ISLAND'
    
    # add both zips and boroughs to map
    zip_borough_web[zip1] = borough1
    zip_borough_web[zip2] = borough2

In [ ]:
unspecified_boroughs_before = complaints_df['borough'].value_counts()['Unspecified']
# replace the boroughs from 311 data with web data by zipcodes from web
def replace_borough(row, zip_dict):
    z = row['incident_zip']
    if z in zip_dict:
        return zip_dict[z]
    else:
        return row['borough']

complaints_df['borough'] = complaints_df.apply(lambda row: replace_borough(row,zip_borough_web), axis=1)

In [ ]:
unspecified_boroughs_after = complaints_df['borough'].value_counts()['Unspecified']
print("Total 'Unspecified' boroughs fixed with web data: {}".format(unspecified_boroughs_before - unspecified_boroughs_after))

### Use311 data to clean
#### Check which zip codes have more than one borough associated with it in the 311 data

In [ ]:
# map zipcodes to boroughs
# dictionary of zipcodes to list of boroughs
zip_borough_311 = {}
count = 0

# if zipcode was seen before but the borough is new, add it to the list of boroughs for that zip
# otherwise create new list with that borough
for row in complaints_df[['borough','incident_zip']].itertuples():
    z = row.incident_zip
    b = row.borough
    
    # attach a new list to an unseen borough 
    if z not in zip_borough_311:
        zip_borough_311[z] = []
        
    # reject if borough exists in list or is unspecified
    if b not in zip_borough_311[z] and row.borough != 'Unspecified':
        zip_borough_311[z].append(b)
        count += 1
        
print("Number of rows with conflicting boroughs: " + str(count))

In [ ]:
# take just the first borough that in the list (could use a count for most frequent borough)
zip_borough_311 =  {k:v[0] for (k,v) in zip_borough_311.items() if len(v) > 0}

In [ ]:
# replace 'Unspecified' boroughs with mapped zipcodes from 311 data duplicates
complaints_df['borough'] = complaints_df.apply(lambda row: replace_borough(row, zip_borough_311), axis=1)

In [ ]:
# remove remaining rows with 'Unspecified' boroughs that could not be determined from other data
complaints_df = complaints_df.query("borough != 'Unspecified'")

print("Unspecified boroughs removed!")

# Top 10 complaint types by borough

In [ ]:
# get the 10 most common overall complaint types
top_10_complaint_types = complaints_df.complaint_type.value_counts()[:10]

print("Number of unique complaint types in all boroughs:\n{}\n".format(complaints_df.complaint_type.nunique()))
print("Top 10 complaint types overall in all boroughs:\n{}".format(top_10_complaint_types))

In [ ]:
# create pivot table using the boroughs as the index and a count of each unique complaint type
complaints_by_borough = complaints_df.pivot_table(index='borough', columns='complaint_type', aggfunc=len)

# get a cross section of a column without a mutlilevel index (shouldn't matter which column is chosen)
complaints_by_borough = complaints_by_borough.xs('city', axis=1, drop_level=True)

In [ ]:
# print just the top 10 complaint types 
complaints_by_borough[top_10_complaint_types.index]

In [ ]:
# Verify previous pivot table results
# get the number of 10 most common overall complaint types PER BOROUGH
"""
# I only ran this to verify the output of the pivot table but it takes a while so...

for borough in complaints_df.borough.unique():
    for complaint in top_10_complaint_types.index:
        count = len(complaints_df[(complaints_df['borough'] == borough) & (complaints_df['complaint_type'] == complaint)])
        print("Borough: {}\tType: {}\tCount: {}".format(borough, complaint, count))
    print("")
"""

# Top 10 complaint types for top 10 most populous zipcodes

In [ ]:
# Get 2010 census population by zipcode 
zipcode_df = pd.read_csv("https://s3.amazonaws.com/SplitwiseBlogJB/2010+Census+Population+By+Zipcode+(ZCTA).csv")

# renaming the columns for easier access
zipcode_df.rename(columns = {'Zip Code ZCTA':'ZCTA', '2010 Census Population':'Population'}, inplace = True)

# Filter the zipcode dataframe to include only NYC zipcodes (the ones in our 311 Service Request dataframe)
zipcode_df = zipcode_df[zipcode_df['ZCTA'].isin(complaints_df.incident_zip.unique())]

In [ ]:
# sort the zipcode dataframe by population in descending order
zipcode_df.sort_values(by='Population', ascending=False, inplace=True)

# print top 10 most populous zip codes
zipcode_df[:10]

In [ ]:
# top 10 most populous zip codes (converted to string for indexing)
top_10_most_populous_zips = [str(x) for x in zipcode_df['ZCTA'][:10].tolist()]

In [ ]:
# create pivot table using the incident_zip as the index and a count of each unique complaint type
complaints_by_zip = complaints_df.pivot_table(index='incident_zip', columns='complaint_type', aggfunc=len)

# get a cross section of a column without a mutlilevel index (shouldn't matter which column is chosen)
complaints_by_zip = complaints_by_zip.xs('borough', axis=1, drop_level=True)

In [ ]:
# print the top 10 complaints in the top 10 most populous zip codes
complaints_by_zip.loc[top_10_most_populous_zips][top_10_complaint_types.index]

# Which boroughs are the biggest "complainers" relative to the size of the population in 2017?
# Calculate a complaint-index that adjusts for population of the borough.

In [ ]:
# data structures to hold calculated data
population_by_borough = []
num_complaints_by_borough = []

# number of complaints by zip code
num_complaints_by_zip = complaints_df['incident_zip'].value_counts().reset_index()
num_complaints_by_zip.rename(columns = {'index':'ZCTA', 'incident_zip':'num_complaints'}, inplace = True)

# iterate over boroughs
for borough in complaints_df.borough.unique():
    # get zips for borough
    zips_of_borough = complaints_df.query('borough == @borough')['incident_zip']
    
    # sum population of each zipcode in the borough
    population = np.sum(zipcode_df[zipcode_df['ZCTA'].isin(zips_of_borough)]['Population'])
    population_by_borough.append( (borough, population) )
    
    # sum number of complaints by zipcode in the borough
    num_complaints = np.sum(num_complaints_by_zip[num_complaints_by_zip['ZCTA'].isin(zips_of_borough)]['num_complaints'])
    num_complaints_by_borough.append( (borough, num_complaints) )

In [ ]:
# the population of each borough
print("Population of each borough:\n{}".format(population_by_borough))
print("Number of complaints for each borough:\n{}".format(num_complaints_by_borough))

In [ ]:
# calculate population-adjusted complaint-index for each borough
# The complaint-index measures the number of complaints in a borough compared to other boroughs in NYC

# calcualate ratio of complaints to population size for each borough
# get the population size of each borough relative to the total population of NYC
# adjusted complaint-index is the ratio of complaints in the borough relative to borough population share in NYC

total_nyc_population = np.sum(int(p[1]) for p in population_by_borough)

complaint_index = []
for i in range(len(population_by_borough)):
    pct_complaints_for_borough = num_complaints_by_borough[i][1]/population_by_borough[i][1]
    relative_population = population_by_borough[i][1]/total_nyc_population
    
    adjusted_complaint_index = pct_complaints_for_borough/relative_population
    complaint_index.append((population_by_borough[i][0], adjusted_complaint_index))

In [ ]:
# sort the list by largest complaint index and print
complaint_index.sort(key=lambda x: x[1], reverse=True)
print("The biggest complainers in descending order using complaint-index:\n{}".format(complaint_index))